In [2]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/2.4.4/libexec")

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer, RegexTokenizer, StopWordsRemover, IDF, MinHashLSH
from pyspark.sql import SQLContext, Row
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.sql.functions import explode, col
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession\
        .builder\
        .appName("test_newwork")\
        .getOrCreate()

spark

In [4]:
sc = spark.sparkContext
sc

<SparkContext master=local[2] appName=test_newwork>

In [5]:
# load all the textfiles 
data=sc.wholeTextFiles("/Users/bharathsurianarayanan/Downloads/cookbook_text/*.txt")

In [6]:
# create a dataframe using the loaded data
df = spark.createDataFrame(data)
# display the dataframe df
df.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|file:/Users/bhara...| 
 
 
  Henrie...|
|file:/Users/bhara...| 
 
 
   Cooki...|
|file:/Users/bhara...| 
 
 
   
   ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 

 
  The ...|
|file:/Users/bhara...| 
 
  
 
 The...|
|file:/Users/bhara...| 
 
 
 
 Brea...|
|file:/Users/bhara...| 
 
 
  
 
 ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 
 
 
  The...|
|file:/Users/bhara...| 
 
 
 
 
...|
|file:/Users/bhara...| 
 
 
 
  The...|
|file:/Users/bhara...| 
 
 
 
 Manu...|
|file:/Users/bhara...| 
 
 
 
 Comm...|
|file:/Users/bhara...| 
 
 
 
 
...|
|file:/Users/bhara...| 
 
 
 
 Favo...|
|file:/Users/bhara...|The White House C...|
|file:/Users/bhara...|The Young House-K...|
|file:/Users/bhara...| 
 
 
 
 The ...|
+--------------------+--------------------+
only showing top 20 r

In [7]:
from pyspark.sql.functions import *
# rename the column 1 to FIle_name
newdf = df.withColumnRenamed("_1", "File_name")
# rename the column 2 to text
newdf = newdf.withColumnRenamed("_2", "text")
newdf.select('File_name').show(2)

+--------------------+
|           File_name|
+--------------------+
|file:/Users/bhara...|
|file:/Users/bhara...|
+--------------------+
only showing top 2 rows



In [8]:
# display yhe schema and the top 20 rows of the newdf
newdf.printSchema()
newdf.show()

root
 |-- File_name: string (nullable = true)
 |-- text: string (nullable = true)

+--------------------+--------------------+
|           File_name|                text|
+--------------------+--------------------+
|file:/Users/bhara...| 
 
 
  Henrie...|
|file:/Users/bhara...| 
 
 
   Cooki...|
|file:/Users/bhara...| 
 
 
   
   ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 

 
  The ...|
|file:/Users/bhara...| 
 
  
 
 The...|
|file:/Users/bhara...| 
 
 
 
 Brea...|
|file:/Users/bhara...| 
 
 
  
 
 ...|
|file:/Users/bhara...| 
 
 
 
 The ...|
|file:/Users/bhara...| 
 
 
 
  The...|
|file:/Users/bhara...| 
 
 
 
 
...|
|file:/Users/bhara...| 
 
 
 
  The...|
|file:/Users/bhara...| 
 
 
 
 Manu...|
|file:/Users/bhara...| 
 
 
 
 Comm...|
|file:/Users/bhara...| 
 
 
 
 
...|
|file:/Users/bhara...| 
 
 
 
 Favo...|
|file:/Users/bhara...|The White House C...|
|file:/Users/bhara...|The Young House-K...|
|file:/Users/bhara...|

In [9]:
# perform processing so as to remove the directory name from the file name
newdf = newdf.withColumn('File_name', regexp_replace('File_name', 'file:/Users/bharathsurianarayanan/Downloads/cookbook_text/', ''))
# tokenize the column text and name it words
tokenizer = Tokenizer(inputCol="text", outputCol="words")
newdf.printSchema()
# newdf.show()
# 
wordsDf = tokenizer.transform(newdf)

vocabSize = 1000000
# convert the text document to vectors using the countVectorizer so as to vectorize the words column
# the vectorized column is now named features
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=vocabSize, minDF=2).fit(wordsDf)
vectorizedDf = cv.transform(wordsDf).select(col("File_name"), col("features"))
vectorizedDf.show()

root
 |-- File_name: string (nullable = true)
 |-- text: string (nullable = true)

+---------+--------------------+
|File_name|            features|
+---------+--------------------+
| henr.txt|(65008,[0,1,2,3,4...|
| cclu.txt|(65008,[0,1,2,3,4...|
| beec.txt|(65008,[0,1,2,3,4...|
| hous.txt|(65008,[0,1,2,3,4...|
| virg.txt|(65008,[0,1,2,3,4...|
| ldnw.txt|(65008,[0,1,2,3,4...|
| conf.txt|(65008,[0,1,2,3,4...|
| brkf.txt|(65008,[0,1,2,3,4...|
| buck.txt|(65008,[0,1,2,3,4...|
| coow.txt|(65008,[0,1,2,3,4...|
| grea.txt|(65008,[0,1,2,3,4...|
| blue.txt|(65008,[0,1,2,3,4...|
| frch.txt|(65008,[0,1,2,3,4...|
| army.txt|(65008,[0,1,2,3,4...|
| comm.txt|(65008,[0,1,2,3,4...|
| bost.txt|(65008,[0,1,2,3,4...|
| favd.txt|(65008,[0,1,2,3,4...|
| whit.txt|(65008,[0,1,2,3,4...|
| youn.txt|(65008,[0,1,2,3,4...|
| neig.txt|(65008,[0,1,2,3,4...|
+---------+--------------------+
only showing top 20 rows



In [10]:
# compute hashes for the vectorized features column, this hash is thus used for comparing text files
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
# build the model for the vectorizeddf
model = mh.fit(vectorizedDf)

In [11]:
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(vectorizedDf).show()

The hashed dataset where hashed values are stored in the column 'hashes':
+---------+--------------------+--------------------+
|File_name|            features|              hashes|
+---------+--------------------+--------------------+
| henr.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| cclu.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| beec.txt|(65008,[0,1,2,3,4...|[[69017.0], [1946...|
| hous.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| virg.txt|(65008,[0,1,2,3,4...|[[178754.0], [192...|
| ldnw.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| conf.txt|(65008,[0,1,2,3,4...|[[178754.0], [692...|
| brkf.txt|(65008,[0,1,2,3,4...|[[178754.0], [192...|
| buck.txt|(65008,[0,1,2,3,4...|[[69017.0], [1946...|
| coow.txt|(65008,[0,1,2,3,4...|[[69017.0], [1946...|
| grea.txt|(65008,[0,1,2,3,4...|[[178754.0], [692...|
| blue.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| frch.txt|(65008,[0,1,2,3,4...|[[178754.0], [192...|
| army.txt|(65008,[0,1,2,3,4...|[[178754.0], [194...|
| comm.t

In [12]:
# set the threshold and then used the built model to calculate the similarity index for all the files
# all files having similarity greater than the threshold will be displayed
threshold = 0.8
comparison=model.approxSimilarityJoin(vectorizedDf, vectorizedDf, threshold).filter("distCol != 0")
comparison.show()

+--------------------+--------------------+------------------+
|            datasetA|            datasetB|           distCol|
+--------------------+--------------------+------------------+
|[bost.txt, (65008...|[lady.txt, (65008...|0.7293906810035842|
|[cclu.txt, (65008...|[hosf.txt, (65008...|0.7473989595838335|
|[prho.txt, (65008...|[amwh.txt, (65008...|0.6818581477139507|
|[time.txt, (65008...|[buck.txt, (65008...|0.7516558232225543|
|[jewi.txt, (65008...|[buck.txt, (65008...|0.7083195043699524|
|[engl.txt, (65008...|[miss.txt, (65008...|0.7498535442296426|
|[lady.txt, (65008...|[notm.txt, (65008...| 0.726773575039219|
|[econ.txt, (65008...|[pcdg.txt, (65008...|0.7564141722663409|
|[favd.txt, (65008...|[fran.txt, (65008...|0.7800912721049629|
|[good.txt, (65008...|[jenn.txt, (65008...|0.7117792195924335|
|[matf.txt, (65008...|[mary.txt, (65008...|0.7983123004409305|
|[miss.txt, (65008...|[wash.txt, (65008...| 0.682782801254844|
|[neig.txt, (65008...|[pres.txt, (65008...|0.7004736051

In [13]:
# number of columns having similarity greater than 0.8
comparison.count()

3249